# Last Away ~ all frequencies: 

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import signal, stats
import mat73
import re
from neurodsp.timefrequency import compute_wavelet_transform
from BCI2kReader import BCI2kReader as b2k
import os
import mne
from tabulate import tabulate
import IPython
import seaborn as sns
import scipy
import joblib
import h5io
import dask.array as da

import statsmodels
from statsmodels import stats
from statsmodels.stats import multitest

# Import required code for visualizing example models
from fooof import FOOOF
from fooof.sim.gen import gen_power_spectrum
from fooof.sim.utils import set_random_seed
from fooof.plts.spectra import plot_spectra
from fooof.plts.annotate import plot_annotated_model
from neurodsp.utils import create_times
from neurodsp.plts.time_series import plot_time_series
from neurodsp.spectral import compute_spectrum, rotate_powerlaw
from neurodsp.plts.spectral import plot_power_spectra



In [ ]:
## Prep paths ##

subject = ''
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}"
preproc_data_dir = f"/home/brooke/pacman/preprocessing/{subject}/ieeg"

In [ ]:
## Load Neural Data

# load
last_away_epochs = mne.read_epochs(f"{preproc_data_dir}/{subject}_bp_raw_clean_last_away_events.fif")

# get good epochs (for behavioral data only)
good_epochs = [i for i,x in enumerate(last_away_epochs.get_annotations_per_epoch()) if not x]
bad_epochs = [i for i,x in enumerate(last_away_epochs.get_annotations_per_epoch()) if  x]

# load behavioral data
last_away_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_last_away_events.csv")

# set info as metadata
last_away_epochs.metadata = last_away_data

# onlt good epochs
last_away_epochs = last_away_epochs[good_epochs]

In [ ]:
## Dictionary of electrode locations ##

# Dictionary mapping ROI to elecs
ROIs = {
    'hc'     :[],
    'ofc'    :[],
    'amyg'   :[],
    'cing'   :[],
    'insula' :[],
    'dlpfc' :[],
    'ec' :[]
}

## prep lists

# primary ROI
hc_list = []
hc_indices = []
hc_names = []
ofc_list = []
ofc_indices = []
ofc_names = []
amyg_list = []
amyg_names = [] 
amyg_indices = []
cing_list = []
cing_names = [] 
cing_indices = []

# control ROI
insula_list = []
insula_names = []  
insula_indices = []
dlpfc_list = []
dlpfc_names = []  
dlpfc_indices = []
ec_list = []
ec_names = []  
ec_indices = []

# exclude bad ROI from list
pairs_long_name = [ch.split('-') for ch in last_away_epochs.info['ch_names']]
bidx = len(last_away_epochs.info['bads']) +1
pairs_name = pairs_long_name[bidx:len(pairs_long_name)]

# sort ROI into lists
for ix in range(0, len(pairs_name)):
    if pairs_name[ix][0] in ROIs['hc'] or pairs_name[ix][1] in ROIs['hc']:
        hc_list.append(last_away_epochs.info['ch_names'][ix + bidx])
        hc_names.append(pairs_name[ix])
        hc_indices.append(ix)
    if pairs_name[ix][0] in ROIs['ofc'] or pairs_name[ix][1] in ROIs['ofc']:
        ofc_list.append(last_away_epochs.info['ch_names'][ix + bidx])
        ofc_names.append(pairs_name[ix])
        ofc_indices.append(ix)
    if pairs_name[ix][0] in ROIs['amyg'] or pairs_name[ix][1] in ROIs['amyg']:
        amyg_list.append(last_away_epochs.info['ch_names'][ix + bidx])       
        amyg_names.append(pairs_name[ix])
        amyg_indices.append(ix)
    if pairs_name[ix][0] in ROIs['cing'] or pairs_name[ix][1] in ROIs['cing']:
        cing_list.append(last_away_epochs.info['ch_names'][ix + bidx])       
        cing_names.append(pairs_name[ix])
        cing_indices.append(ix)
        
    # control roi
    if pairs_name[ix][0] in ROIs['insula'] or pairs_name[ix][1] in ROIs['insula']:
        insula_list.append(last_away_epochs.info['ch_names'][ix + bidx])       
        insula_names.append(pairs_name[ix])
        insula_indices.append(ix)
    if pairs_name[ix][0] in ROIs['dlpfc'] or pairs_name[ix][1] in ROIs['dlpfc']:
        dlpfc_list.append(last_away_epochs.info['ch_names'][ix + bidx])       
        dlpfc_names.append(pairs_name[ix])
        dlpfc_indices.append(ix)
    if pairs_name[ix][0] in ROIs['ec'] or pairs_name[ix][1] in ROIs['ec']:
        ec_list.append(last_away_epochs.info['ch_names'][ix + bidx])       
        ec_names.append(pairs_name[ix])
        ec_indices.append(ix)        
        

In [ ]:
## funcions

def MM_compute_TFR(epochs, freqs, n_cycles, baseline, zscore=True, workers = 8):
    """ function to compute TFR via Morlet wavelets
    
    epochs:                     MNE epoch object with channels of interest
    freqs:                      list of frequencies, should be log spaced
    n_cycles:                   number of cycles, adjust with freqs to balance temporal and frequency resolution
    baseline:                   tuple that specifies the time in seconds that baseline should be calculated on 
                                (should be 1-2s smaller than epoch TFR was calculated on to exclude artificats on morlet waves
    zscore:                     boolean, if True zscores within trial and frequency band across the time window specified in baseline
    workers:                    number of threads to use while calculating TFR
    """
    print('computing TFR')
    TFR = mne.time_frequency.tfr_morlet(epochs,freqs,n_cycles,return_itc=False,
                                            average=False,  n_jobs = workers)
    print('log transforming')
    TFR.data = da.from_array(TFR.data, chunks="auto")  # Convert the data to a Dask array
    TFR.data = da.log(TFR.data)  # Compute the element-wise log

    if zscore:
        print('z-scoring to baseline')
        bix = [a and b for a, b in zip(TFR.times >= baseline[0], TFR.times <= baseline[1])]
        bmean = TFR.data[:, :, :, bix].mean(axis=3, keepdims=True).mean(axis=0, keepdims=True)
        bstd = TFR.data[:, :, :, bix].std(axis=3, keepdims=True).std(axis=0, keepdims=True)
        TFR.data = (TFR.data - bmean) / bstd

    TFR.data = TFR.data.compute()  # Convert the Dask array back to a NumPy array
    return TFR

def extract_freqs(lower_freq, higher_freq, freq_band, subdir, ROI, label, TFR, trials):
    """ function to extract and average the across the freqs within a given band and save out to csvs
    step is calculated based on getting ~4 samples per frequency cycle
    
    lower_freq, higher_freq:    non inclusive lower and upper bounds of the band
    freq_band:                  band name, as a string
    subdir:                     dir in sub/ieeg/ that specifies the time locking
    ROI:                        region name, as a string
    label:                      label, eg ghost, no ghost, choice locked etc, as a string
    TFR:                        MNE TFR object
    """
    
    # calculate step, ## note I made this up, but it seems reasonable?
    step = {
        'delta'      : int(np.floor(ghost_TFR.info['sfreq']/(2*4))),
        'theta'      : int(np.floor(ghost_TFR.info['sfreq']/(5*4))),
        'alpha'      : int(np.floor(ghost_TFR.info['sfreq']/(11*4))),
        'beta'       : int(np.floor(ghost_TFR.info['sfreq']/(22*4))),
        'gamma'      : int(np.floor(ghost_TFR.info['sfreq']/(50*4))),
        'hfa'        : int(np.floor(ghost_TFR.info['sfreq']/(110*4)))
    }
    
    # check if it needs to be calculated with subbands
    if freq_band == 'gamma' or freq_band == 'hfa':
        
        if ghost_TFR.info['sfreq'] < 1000:
                                raise Exception('Sampling Rate is below 1000, should calculate gamma or HFA') 
        
        subband_dict = {
            'gamma'    : [(30, 40), (35, 45), (40, 50), (45, 55), (50, 60), (55, 65), (60, 70)],
            'hfa'      : [(70, 90), (80, 100), (90, 110), (100, 120), (110, 130), (120, 140), (130, 150)]
        }
        
        for chix in range(len(TFR.ch_names)):
            subb_trial_power = []
            for subb in subband_dict[freq_band]:
                fidx = np.where((freqs > subb[0]) & (freqs < subb[1]))[0]
                subb_trial_power.append(TFR.data[:, chix, fidx, :].mean(axis=1))
            trial_power = np.mean(subb_trial_power, axis = 0)
            channel_df = pd.DataFrame(trial_power[:, ::step[freq_band]])
            channel_df["trial"] = trials
            channel_df.to_csv(f"{preproc_data_dir}/{subdir}/{TFR.ch_names[chix]}_{ROI}_trial_{freq_band}_{label}.csv")
    
    else:
        fidx = np.where((freqs > lower_freq) & (freqs < higher_freq))[0]
        for chix in range(len(TFR.ch_names)):
            trial_power = TFR.data[:, chix, fidx, :].mean(axis=1)
            channel_df = pd.DataFrame(trial_power[:, ::step[freq_band]])
            channel_df["trial"] = trials
            channel_df.to_csv(f"{preproc_data_dir}/{subdir}/{TFR.ch_names[chix]}_{ROI}_trial_{freq_band}_{label}.csv")

In [ ]:
# Set frequencies

freqs = np.logspace(start = np.log10(1), stop = np.log10(150), num = 80, base = 10, endpoint = True)
n_cycles = np.logspace(np.log10(2), np.log10(30), base = 10, num = 80)

# formulas to check bandwidth and time bin
band_width = (freqs / n_cycles) * 2
time_bin = n_cycles / freqs / np.pi
print(freqs)
print(n_cycles)
print(time_bin)
print(band_width)


# Main Regions of Interest

## Hippocampus

In [ ]:
hc_list

In [ ]:
# only ROI of interest
away_roi = last_away_epochs.copy().pick_channels(hc_list)


In [ ]:
# Resample to 1000
if away_roi.info['sfreq'] > 1000:
    away_roi= away_roi.resample(1000)


In [ ]:
# compute TRF
away_TFR = MM_compute_TFR(away_roi,freqs,n_cycles,baseline = (-2,2), zscore=True)


In [ ]:
away_TFR.crop(tmin = -2, tmax = 2)

In [ ]:
# separate ghost and no ghost trials

ghost_TFR = away_TFR["TrialType <= 16"]
no_ghost_TFR = away_TFR["TrialType > 16"]

In [ ]:
# save TFRs

ghost_TFR.save(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/last_away/hc_ghost-tfr.h5", overwrite = True)
no_ghost_TFR.save(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/last_away/hc_noghost-tfr.h5", overwrite = True)

# ghost_TFR = mne.time_frequency.read_tfrs(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/last_away/hc_ghost-tfr.h5")
# no_ghost_TFR = mne.time_frequency.read_tfrs(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/last_away/hc_noghost-tfr.h5")
# ghost_TFR = ghost_TFR[0]
# no_ghost_TFR = no_ghost_TFR[0]

In [ ]:
plt.rcParams['figure.figsize'] = [22, 20]
plt.rcParams.update({'font.size': 18})

fig, ax = plt.subplots(figsize = (22, 20))
i = ax.imshow(ghost_TFR.data.mean(axis = 0).mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -1, vmax = 1)
ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
ax.set_yticklabels(np.round(ghost_TFR.freqs))
bar = plt.colorbar(i)
ax.set_title(f"{subject}, Average Hippocampus, Conflict Trials, n={len(hc_list)}", fontsize=22, fontweight = 'bold')
fig.savefig(f'figures/{subject}_average_hc_conflict.png', dpi=600)
fig.show()


fig, ax = plt.subplots(figsize = (22, 20))
i = ax.imshow(no_ghost_TFR.data.mean(axis = 0).mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -1, vmax = 1)
ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
ax.set_yticklabels(np.round(ghost_TFR.freqs))
bar = plt.colorbar(i)
ax.set_title(f"{subject}, Average Hippocampus, No-Conflict Trials, n={len(hc_list)}", fontsize=22, fontweight = 'bold')
fig.savefig(f'figures/{subject}_average_hc_noconflict.png', dpi=600)
fig.show()


## Individual Channels

In [ ]:
plt.rcParams['figure.figsize'] = [22, 20]

chix = 0
for ch in ghost_TFR.ch_names:

    fig, ax = plt.subplots(figsize = (22, 20))
    i = ax.imshow(ghost_TFR.data[:, chix, :, :].mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -2.5, vmax = 2.5)
    ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
    ax.set_yticklabels(np.round(ghost_TFR.freqs))
    bar = plt.colorbar(i)
    ax.set_title(f"{ch}:  Conflict Trials")
    fig.show()

    fig, ax = plt.subplots(figsize = (22, 20))
    i = ax.imshow(no_ghost_TFR.data[:, chix, :, :].mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -2.5, vmax = 2.5)
    ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
    ax.set_yticklabels(np.round(ghost_TFR.freqs))
    bar = plt.colorbar(i)
    ax.set_title(f"{ch}:  No Conflict Trials")
    fig.show()
    
    chix = chix + 1

## Export freq data locked to turn around time to csvs

In [ ]:
# delta
extract_freqs(1, 3, "delta", "last_away", "hc", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(1, 3, "delta", "last_away", "hc", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# theta
extract_freqs(3, 8, "theta", "last_away", "hc", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(3, 8, "theta", "last_away", "hc", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# alpha
extract_freqs(8, 13, "alpha", "last_away", "hc", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(8, 13, "alpha", "last_away", "hc", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# beta
extract_freqs(13, 30, "beta", "last_away", "hc", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(13, 30, "beta", "last_away", "hc", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# gamma
extract_freqs(30, 70, "gamma", "last_away", "hc", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(30, 70, "gamma", "last_away", "hc", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# HFA
extract_freqs(70, 150, "hfa", "last_away", "hc", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(70, 150, "hfa", "last_away", "hc", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))

# OFC

In [ ]:
ofc_list

In [ ]:
away_roi = last_away_epochs.copy().pick_channels(ofc_list)

In [ ]:
# Resample to 1000
if away_roi.info['sfreq'] > 1000:
    away_roi= away_roi.resample(1000)


In [ ]:
# compute TRF
away_TFR = MM_compute_TFR(away_roi,freqs,n_cycles,baseline = (-2,2), zscore=True)


In [ ]:
away_TFR.crop(tmin = -2, tmax = 2)

In [ ]:
# separate ghost and no ghost trials

ghost_TFR = away_TFR["TrialType <= 16"]
no_ghost_TFR = away_TFR["TrialType > 16"]

In [ ]:
# save TFRs

ghost_TFR.save(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/last_away/ofc_ghost-tfr.h5")
no_ghost_TFR.save(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/last_away/ofc_noghost-tfr.h5")

# ghost_TFR = mne.time_frequency.read_tfrs(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/last_away/ofc_ghost-tfr.h5")
# no_ghost_TFR = mne.time_frequency.read_tfrs(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/last_away/ofc_noghost-tfr.h5")
# ghost_TFR = ghost_TFR[0]
# no_ghost_TFR = no_ghost_TFR[0]

In [ ]:
plt.rcParams['figure.figsize'] = [22, 20]
plt.rcParams.update({'font.size': 18})

fig, ax = plt.subplots(figsize = (22, 20))
i = ax.imshow(ghost_TFR.data.mean(axis = 0).mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -1, vmax = 1)
ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
ax.set_yticklabels(np.round(ghost_TFR.freqs))
bar = plt.colorbar(i)
ax.set_title(f"{subject}, Average OFC, Conflict Trials, n={len(ofc_list)}", fontsize=22, fontweight = 'bold')
fig.savefig(f'figures/{subject}_average_ofc_conflict.png', dpi=600)
fig.show()

fig, ax = plt.subplots(figsize = (22, 20))
i = ax.imshow(no_ghost_TFR.data.mean(axis = 0).mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -1, vmax = 1)
ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
ax.set_yticklabels(np.round(ghost_TFR.freqs))
bar = plt.colorbar(i)
ax.set_title(f"{subject}, Average OFC, No-Conflict Trials, n={len(ofc_list)}", fontsize=22, fontweight = 'bold')
fig.savefig(f'figures/{subject}_average_ofc_noconflict.png', dpi=600)
fig.show()



### Individual Channels

In [ ]:
plt.rcParams['figure.figsize'] = [22, 20]

chix = 0
for ch in ghost_TFR.ch_names:

    fig, ax = plt.subplots(figsize = (22, 20))
    i = ax.imshow(ghost_TFR.data[:, chix, :, :].mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -2.5, vmax = 2.5)
    ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
    ax.set_yticklabels(np.round(ghost_TFR.freqs))
    bar = plt.colorbar(i)
    ax.set_title(f"{ch}:  Conflict Trials")
    fig.show()

    fig, ax = plt.subplots(figsize = (22, 20))
    i = ax.imshow(no_ghost_TFR.data[:, chix, :, :].mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -2.5, vmax = 2.5)
    ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
    ax.set_yticklabels(np.round(ghost_TFR.freqs))
    bar = plt.colorbar(i)
    ax.set_title(f"{ch}:  No Conflict Trials")
    fig.show()
    
    chix = chix + 1

### Export freq data locked to turn around time to csvs

In [ ]:
# delta
extract_freqs(1, 3, "delta", "last_away", "ofc", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(1, 3, "delta", "last_away", "ofc", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# theta
extract_freqs(3, 8, "theta", "last_away", "ofc", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(3, 8, "theta", "last_away", "ofc", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# alpha
extract_freqs(8, 13, "alpha", "last_away", "ofc", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(8, 13, "alpha", "last_away", "ofc", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# beta
extract_freqs(13, 30, "beta", "last_away", "ofc", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(13, 30, "beta", "last_away", "ofc", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# gamma
extract_freqs(30, 70, "gamma", "last_away", "ofc", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(30, 70, "gamma", "last_away", "ofc", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# HFA
extract_freqs(70, 150, "hfa", "last_away", "ofc", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(70, 150, "hfa", "last_away", "ofc", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))

# Amygdala

In [ ]:
amyg_list

In [ ]:
away_roi = last_away_epochs.copy().pick_channels(amyg_list)

In [ ]:
# Resample to 1000
if away_roi.info['sfreq'] > 1000:
    away_roi= away_roi.resample(1000)


In [ ]:
# compute TRF
away_TFR = MM_compute_TFR(away_roi,freqs,n_cycles,baseline = (-2,2), zscore=True)


In [ ]:
away_TFR.crop(tmin = -2, tmax = 2)

In [ ]:
# separate ghost and no ghost trials

ghost_TFR = away_TFR["TrialType <= 16"]
no_ghost_TFR = away_TFR["TrialType > 16"]

In [ ]:
# save TFRs

ghost_TFR.save(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/last_away/amyg_ghost-tfr.h5")
no_ghost_TFR.save(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/last_away/amyg_noghost-tfr.h5")


# ghost_TFR = mne.time_frequency.read_tfrs(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/last_away/amyg_ghost-tfr.h5")
# no_ghost_TFR = mne.time_frequency.read_tfrs(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/last_away/amyg_noghost-tfr.h5")
# ghost_TFR = ghost_TFR[0]
# no_ghost_TFR = no_ghost_TFR[0]

In [ ]:
plt.rcParams['figure.figsize'] = [22, 20]
plt.rcParams.update({'font.size': 18})

fig, ax = plt.subplots(figsize = (22, 20))
i = ax.imshow(ghost_TFR.data.mean(axis = 0).mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -1, vmax = 1)
ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
ax.set_yticklabels(np.round(ghost_TFR.freqs))
bar = plt.colorbar(i)
ax.set_title(f"{subject}, Average Amygdala, Conflict Trials, n={len(amyg_list)}", fontsize=22, fontweight = 'bold')
fig.savefig(f'figures/{subject}_average_amyg_conflict.png', dpi=600)
fig.show()


fig, ax = plt.subplots(figsize = (22, 20))
i = ax.imshow(no_ghost_TFR.data.mean(axis = 0).mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -1, vmax = 1)
ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
ax.set_yticklabels(np.round(ghost_TFR.freqs))
bar = plt.colorbar(i)
ax.set_title(f"{subject}, Average Amygdala, No-Conflict Trials, n={len(amyg_list)}", fontsize=22, fontweight = 'bold')
fig.savefig(f'figures/{subject}_average_amyg_noconflict.png', dpi=600)
fig.show()

### Individual Channels

In [ ]:
plt.rcParams['figure.figsize'] = [22, 20]

chix = 0
for ch in ghost_TFR.ch_names:

    fig, ax = plt.subplots(figsize = (22, 20))
    i = ax.imshow(ghost_TFR.data[:, chix, :, :].mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -2.5, vmax = 2.5)
    ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
    ax.set_yticklabels(np.round(ghost_TFR.freqs))
    bar = plt.colorbar(i)
    ax.set_title(f"{ch}:  Conflict Trials")
    fig.show()

    fig, ax = plt.subplots(figsize = (22, 20))
    i = ax.imshow(no_ghost_TFR.data[:, chix, :, :].mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -2.5, vmax = 2.5)
    ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
    ax.set_yticklabels(np.round(ghost_TFR.freqs))
    bar = plt.colorbar(i)
    ax.set_title(f"{ch}:  No Conflict Trials")
    fig.show()
    
    chix = chix + 1

### Export freq data locked to turn around time to csvs

In [ ]:
# delta
extract_freqs(1, 3, "delta", "last_away", "amyg", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(1, 3, "delta", "last_away", "amyg", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# theta
extract_freqs(3, 8, "theta", "last_away", "amyg", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(3, 8, "theta", "last_away", "amyg", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# alpha
extract_freqs(8, 13, "alpha", "last_away", "amyg", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(8, 13, "alpha", "last_away", "amyg", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# beta
extract_freqs(13, 30, "beta", "last_away", "amyg", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(13, 30, "beta", "last_away", "amyg", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# gamma
extract_freqs(30, 70, "gamma", "last_away", "amyg", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(30, 70, "gamma", "last_away", "amyg", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# HFA
extract_freqs(70, 150, "hfa", "last_away", "amyg", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(70, 150, "hfa", "last_away", "amyg", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))

## Cingulate

In [ ]:
cing_list

In [ ]:
away_roi = last_away_epochs.copy().pick_channels(cing_list)

In [ ]:
# Resample to 1000
if away_roi.info['sfreq'] > 1000:
    away_roi= away_roi.resample(1000)


In [ ]:
# compute TRF
away_TFR = MM_compute_TFR(away_roi,freqs,n_cycles,baseline = (-2,2), zscore=True)


In [ ]:
away_TFR.crop(tmin = -2, tmax = 2)

In [ ]:
# separate ghost and no ghost trials

ghost_TFR = away_TFR["TrialType <= 16"]
no_ghost_TFR = away_TFR["TrialType > 16"]

In [ ]:
# save TFRs

ghost_TFR.save(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/last_away/cing_ghost-tfr.h5")
no_ghost_TFR.save(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/last_away/cing_noghost-tfr.h5")

# ghost_TFR = mne.time_frequency.read_tfrs(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/last_away/cing_ghost-tfr.h5")
# no_ghost_TFR = mne.time_frequency.read_tfrs(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/last_away/cing_noghost-tfr.h5")
# ghost_TFR = ghost_TFR[0]
# no_ghost_TFR = no_ghost_TFR[0]

In [ ]:
plt.rcParams['figure.figsize'] = [22, 20]
plt.rcParams.update({'font.size': 18})

fig, ax = plt.subplots(figsize = (22, 20))
i = ax.imshow(ghost_TFR.data.mean(axis = 0).mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -1, vmax = 1)
ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
ax.set_yticklabels(np.round(ghost_TFR.freqs))
bar = plt.colorbar(i)
ax.set_title(f"{subject}, Average Cingulate, Conflict Trials, n={len(cing_list)}", fontsize=22, fontweight = 'bold')
fig.savefig(f'figures/{subject}_average_cing_conflict.png', dpi=600)
fig.show()


fig, ax = plt.subplots(figsize = (22, 20))
i = ax.imshow(no_ghost_TFR.data.mean(axis = 0).mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -1, vmax = 1)
ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
ax.set_yticklabels(np.round(ghost_TFR.freqs))
bar = plt.colorbar(i)
ax.set_title(f"{subject}, Average Cingulate, No-Conflict Trials, n={len(cing_list)}", fontsize=22, fontweight = 'bold')
fig.savefig(f'figures/{subject}_average_cing_noconflict.png', dpi=600)
fig.show()


### Individual Channels

In [ ]:
plt.rcParams['figure.figsize'] = [22, 20]

chix = 0
for ch in ghost_TFR.ch_names:

    fig, ax = plt.subplots(figsize = (22, 20))
    i = ax.imshow(ghost_TFR.data[:, chix, :, :].mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -2.5, vmax = 2.5)
    ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
    ax.set_yticklabels(np.round(ghost_TFR.freqs))
    bar = plt.colorbar(i)
    ax.set_title(f"{ch}:  Conflict Trials")
    fig.show()

    fig, ax = plt.subplots(figsize = (22, 20))
    i = ax.imshow(no_ghost_TFR.data[:, chix, :, :].mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -2.5, vmax = 2.5)
    ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
    ax.set_yticklabels(np.round(ghost_TFR.freqs))
    bar = plt.colorbar(i)
    ax.set_title(f"{ch}:  No Conflict Trials")
    fig.show()
    
    chix = chix + 1

### Export freq data locked to turn around time to csvs

In [ ]:
# delta
extract_freqs(1, 3, "delta", "last_away", "cing", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(1, 3, "delta", "last_away", "cing", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# theta
extract_freqs(3, 8, "theta", "last_away", "cing", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(3, 8, "theta", "last_away", "cing", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# alpha
extract_freqs(8, 13, "alpha", "last_away", "cing", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(8, 13, "alpha", "last_away", "cing", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# beta
extract_freqs(13, 30, "beta", "last_away", "cing", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(13, 30, "beta", "last_away", "cing", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# gamma
extract_freqs(30, 70, "gamma", "last_away", "cing", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(30, 70, "gamma", "last_away", "cing", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# HFA
extract_freqs(70, 150, "hfa", "last_away", "cing", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(70, 150, "hfa", "last_away", "cing", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))

# Control Regions

## Insula

In [ ]:
insula_list

In [ ]:
away_roi = last_away_epochs.copy().pick_channels(insula_list)

In [ ]:
# Resample to 1000
if away_roi.info['sfreq'] > 1000:
    away_roi= away_roi.resample(1000)


In [ ]:
# compute TRF
away_TFR = MM_compute_TFR(away_roi,freqs,n_cycles,baseline = (-2,2), zscore=True)


In [ ]:
away_TFR.crop(tmin = -2, tmax = 2)

In [ ]:
# separate ghost and no ghost trials

ghost_TFR = away_TFR["TrialType <= 16"]
no_ghost_TFR = away_TFR["TrialType > 16"]

In [ ]:
# save TFRs

ghost_TFR.save(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/last_away/insula_ghost-tfr.h5")
no_ghost_TFR.save(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/last_away/insula_noghost-tfr.h5")

# ghost_TFR = mne.time_frequency.read_tfrs(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/last_away/insula_ghost-tfr.h5")
# no_ghost_TFR = mne.time_frequency.read_tfrs(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/last_away/insula_noghost-tfr.h5")
# ghost_TFR = ghost_TFR[0]
# no_ghost_TFR = no_ghost_TFR[0]

In [ ]:
plt.rcParams['figure.figsize'] = [22, 20]
plt.rcParams.update({'font.size': 18})

fig, ax = plt.subplots(figsize = (22, 20))
i = ax.imshow(ghost_TFR.data.mean(axis = 0).mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -1, vmax = 1)
ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
ax.set_yticklabels(np.round(ghost_TFR.freqs))
bar = plt.colorbar(i)
ax.set_title(f"{subject}, Average Insula, Conflict Trials, n={len(insula_list)}", fontsize=22, fontweight = 'bold')
fig.savefig(f'figures/{subject}_average_insula_conflict.png', dpi=600)
fig.show()


fig, ax = plt.subplots(figsize = (22, 20))
i = ax.imshow(no_ghost_TFR.data.mean(axis = 0).mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -1, vmax = 1)
ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
ax.set_yticklabels(np.round(ghost_TFR.freqs))
bar = plt.colorbar(i)
ax.set_title(f"{subject}, Average Insula, No-Conflict Trials, n={len(insula_list)}", fontsize=22, fontweight = 'bold')
fig.savefig(f'figures/{subject}_average_insula_noconflict.png', dpi=600)
fig.show()


### Individual Channels

In [ ]:
plt.rcParams['figure.figsize'] = [22, 20]

chix = 0
for ch in ghost_TFR.ch_names:

    fig, ax = plt.subplots(figsize = (22, 20))
    i = ax.imshow(ghost_TFR.data[:, chix, :, :].mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -2.5, vmax = 2.5)
    ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
    ax.set_yticklabels(np.round(ghost_TFR.freqs))
    bar = plt.colorbar(i)
    ax.set_title(f"{ch}:  Conflict Trials")
    fig.show()

    fig, ax = plt.subplots(figsize = (22, 20))
    i = ax.imshow(no_ghost_TFR.data[:, chix, :, :].mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -2.5, vmax = 2.5)
    ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
    ax.set_yticklabels(np.round(ghost_TFR.freqs))
    bar = plt.colorbar(i)
    ax.set_title(f"{ch}:  No Conflict Trials")
    fig.show()
    
    chix = chix + 1

### Export freq data locked to turn around time to csvs

In [ ]:
# delta
extract_freqs(1, 3, "delta", "last_away", "insula", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(1, 3, "delta", "last_away", "insula", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# theta
extract_freqs(3, 8, "theta", "last_away", "insula", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(3, 8, "theta", "last_away", "insula", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# alpha
extract_freqs(8, 13, "alpha", "last_away", "insula", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(8, 13, "alpha", "last_away", "insula", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# beta
extract_freqs(13, 30, "beta", "last_away", "insula", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(13, 30, "beta", "last_away", "insula", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# gamma
extract_freqs(30, 70, "gamma", "last_away", "insula", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(30, 70, "gamma", "last_away", "insula", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# HFA
extract_freqs(70, 150, "hfa", "last_away", "insula", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(70, 150, "hfa", "last_away", "insula", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))

## dlPFC

In [ ]:
dlpfc_list

In [ ]:
away_roi = last_away_epochs.copy().pick_channels(dlpfc_list)

In [ ]:
# Resample to 1000
if away_roi.info['sfreq'] > 1000:
    away_roi= away_roi.resample(1000)


In [ ]:
# compute TRF
away_TFR = MM_compute_TFR(away_roi,freqs,n_cycles,baseline = (-2,2), zscore=True)


In [ ]:
away_TFR.crop(tmin = -2, tmax = 2)

In [ ]:
# separate ghost and no ghost trials

ghost_TFR = away_TFR["TrialType <= 16"]
no_ghost_TFR = away_TFR["TrialType > 16"]

In [ ]:
# save TFRs

ghost_TFR.save(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/last_away/dlpfc_ghost-tfr.h5")
no_ghost_TFR.save(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/last_away/dlpfc_noghost-tfr.h5")

# ghost_TFR = mne.time_frequency.read_tfrs(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/last_away/dlpfc_ghost-tfr.h5")
# no_ghost_TFR = mne.time_frequency.read_tfrs(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/last_away/dlpfc_noghost-tfr.h5")
# ghost_TFR = ghost_TFR[0]
# no_ghost_TFR = no_ghost_TFR[0]

In [ ]:
plt.rcParams['figure.figsize'] = [22, 20]
plt.rcParams.update({'font.size': 18})

fig, ax = plt.subplots(figsize = (22, 20))
i = ax.imshow(ghost_TFR.data.mean(axis = 0).mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -1, vmax = 1)
ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
ax.set_yticklabels(np.round(ghost_TFR.freqs))
bar = plt.colorbar(i)
ax.set_title(f"{subject}, Average dlPFC, Conflict Trials, n={len(dlpfc_list)}", fontsize=22, fontweight = 'bold')
fig.savefig(f'figures/{subject}_average_dlpfc_conflict.png', dpi=600)
fig.show()


fig, ax = plt.subplots(figsize = (22, 20))
i = ax.imshow(no_ghost_TFR.data.mean(axis = 0).mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -1, vmax = 1)
ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
ax.set_yticklabels(np.round(ghost_TFR.freqs))
bar = plt.colorbar(i)
ax.set_title(f"{subject}, Average dlPFC, No-Conflict Trials, n={len(dlpfc_list)}", fontsize=22, fontweight = 'bold')
fig.savefig(f'figures/{subject}_average_dlpfc_noconflict.png', dpi=600)
fig.show()


### Individual Channels

In [ ]:
plt.rcParams['figure.figsize'] = [22, 20]

chix = 0
for ch in ghost_TFR.ch_names:

    fig, ax = plt.subplots(figsize = (22, 20))
    i = ax.imshow(ghost_TFR.data[:, chix, :, :].mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -2.5, vmax = 2.5)
    ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
    ax.set_yticklabels(np.round(ghost_TFR.freqs))
    bar = plt.colorbar(i)
    ax.set_title(f"{ch}:  Conflict Trials")
    fig.show()

    fig, ax = plt.subplots(figsize = (22, 20))
    i = ax.imshow(no_ghost_TFR.data[:, chix, :, :].mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -2.5, vmax = 2.5)
    ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
    ax.set_yticklabels(np.round(ghost_TFR.freqs))
    bar = plt.colorbar(i)
    ax.set_title(f"{ch}:  No Conflict Trials")
    fig.show()
    
    chix = chix + 1

### Export freq data locked to turn around time to csvs

In [ ]:
# delta
extract_freqs(1, 3, "delta", "last_away", "dlpfc", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(1, 3, "delta", "last_away", "dlpfc", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# theta
extract_freqs(3, 8, "theta", "last_away", "dlpfc", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(3, 8, "theta", "last_away", "dlpfc", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# alpha
extract_freqs(8, 13, "alpha", "last_away", "dlpfc", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(8, 13, "alpha", "last_away", "dlpfc", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# beta
extract_freqs(13, 30, "beta", "last_away", "dlpfc", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(13, 30, "beta", "last_away", "dlpfc", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# gamma
extract_freqs(30, 70, "gamma", "last_away", "dlpfc", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(30, 70, "gamma", "last_away", "dlpfc", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# HFA
extract_freqs(70, 150, "hfa", "last_away", "dlpfc", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(70, 150, "hfa", "last_away", "dlpfc", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))

## Entorhinal Cortex

In [ ]:
ec_list

In [ ]:
away_roi = last_away_epochs.copy().pick_channels(ec_list)

In [ ]:
# Resample to 1000
if away_roi.info['sfreq'] > 1000:
    away_roi= away_roi.resample(1000)


In [ ]:
# compute TRF
away_TFR = MM_compute_TFR(away_roi,freqs,n_cycles,baseline = (-2,2), zscore=True)


In [ ]:
away_TFR.crop(tmin = -2, tmax = 2)

In [ ]:
# separate ghost and no ghost trials

ghost_TFR = away_TFR["TrialType <= 16"]
no_ghost_TFR = away_TFR["TrialType > 16"]

In [ ]:
# save TFRs

ghost_TFR.save(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/last_away/ec_ghost-tfr.h5")
no_ghost_TFR.save(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/last_away/ec_noghost-tfr.h5")

# ghost_TFR = mne.time_frequency.read_tfrs(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/last_away/e_ghost-tfr.h5")
# no_ghost_TFR = mne.time_frequency.read_tfrs(f"/home/brooke/knight_server/remote/bstavel/pacman/preprocessing/{subject}/ieeg/last_away/e_noghost-tfr.h5")
# ghost_TFR = ghost_TFR[0]
# no_ghost_TFR = no_ghost_TFR[0]

In [ ]:
plt.rcParams['figure.figsize'] = [22, 20]
plt.rcParams.update({'font.size': 18})

fig, ax = plt.subplots(figsize = (22, 20))
i = ax.imshow(ghost_TFR.data.mean(axis = 0).mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -1, vmax = 1)
ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
ax.set_yticklabels(np.round(ghost_TFR.freqs))
bar = plt.colorbar(i)
ax.set_title(f"{subject}, Average EC, Conflict Trials, n={len(ec_list)}", fontsize=22, fontweight = 'bold')
fig.savefig(f'figures/{subject}_average_ec_conflict.png', dpi=600)
fig.show()


fig, ax = plt.subplots(figsize = (22, 20))
i = ax.imshow(no_ghost_TFR.data.mean(axis = 0).mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -1, vmax = 1)
ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
ax.set_yticklabels(np.round(ghost_TFR.freqs))
bar = plt.colorbar(i)
ax.set_title(f"{subject}, Average EC, No-Conflict Trials, n={len(ec_list)}", fontsize=22, fontweight = 'bold')
fig.savefig(f'figures/{subject}_average_ec_noconflict.png', dpi=600)
fig.show()


### Individual Channels

In [ ]:
plt.rcParams['figure.figsize'] = [22, 20]

chix = 0
for ch in ghost_TFR.ch_names:

    fig, ax = plt.subplots(figsize = (22, 20))
    i = ax.imshow(ghost_TFR.data[:, chix, :, :].mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -2.5, vmax = 2.5)
    ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
    ax.set_yticklabels(np.round(ghost_TFR.freqs))
    bar = plt.colorbar(i)
    ax.set_title(f"{ch}:  Conflict Trials")
    fig.show()

    fig, ax = plt.subplots(figsize = (22, 20))
    i = ax.imshow(no_ghost_TFR.data[:, chix, :, :].mean(axis = 0), cmap = 'RdBu_r', interpolation="none", origin="lower", aspect = 'auto', extent=[-2, 2, freqs[0], freqs[-1]], vmin = -2.5, vmax = 2.5)
    ax.set_yticks(np.linspace(np.min(ghost_TFR.freqs),np.max(ghost_TFR.freqs),len(ghost_TFR.freqs)))
    ax.set_yticklabels(np.round(ghost_TFR.freqs))
    bar = plt.colorbar(i)
    ax.set_title(f"{ch}:  No Conflict Trials")
    fig.show()
    
    chix = chix + 1

### Export freq data locked to turn around time to csvs

In [ ]:
# delta
extract_freqs(1, 3, "delta", "last_away", "ec", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(1, 3, "delta", "last_away", "ec", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# theta
extract_freqs(3, 8, "theta", "last_away", "ec", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(3, 8, "theta", "last_away", "ec", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# alpha
extract_freqs(8, 13, "alpha", "last_away", "ec", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(8, 13, "alpha", "last_away", "ec", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# beta
extract_freqs(13, 30, "beta", "last_away", "ec", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(13, 30, "beta", "last_away", "ec", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# gamma
extract_freqs(30, 70, "gamma", "last_away", "ec", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(30, 70, "gamma", "last_away", "ec", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
# HFA
extract_freqs(70, 150, "hfa", "last_away", "ec", "away_locked_ghost", ghost_TFR, ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))
extract_freqs(70, 150, "hfa", "last_away", "ec", "away_locked_noghost", no_ghost_TFR, no_ghost_TFR.metadata.neural_trial_numeric.to_numpy(copy = True))